In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Dependencies
import requests
import pymongo
import dns
import datetime
from datetime import datetime
from dateutil import parser
import pytz
from django.utils import timezone
from time import sleep
import random
import json
import config
import requests
from flask_cors import CORS
import os
import csv


# Project defined dependencies
import config
from pprint import pprint
# import cl_parser


In [3]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(config.mongo_conn)

In [4]:
# Define database and collection
db = client.getitdone_db
SR_collection = db.sandiego
summary_collection = db.summary_counts

In [5]:
for SR in SR_collection.find():
    print(SR['service_request_id'])
    break

In [ ]:
summary_collection.count_documents({'year':2021})

In [ ]:
summary_collection.count_documents({'year':2021})

In [7]:
for summary_obj in summary_collection.find():
    print(summary_obj)
    break

In [ ]:
currentMonth = datetime.now().month
print(currentMonth)
type(currentMonth)

In [ ]:

local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime("2021-" + str(currentMonth) + "-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime("2021-" + str(currentMonth + 1) + "-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

 
print(dt_start_utc, dt_end_utc)

In [ ]:
year = 2020
local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime(str(year) + "-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime(str(year + 1) +"-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

filter_requested ={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}
filter_closed ={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }, 'status':"Closed"
}


total_requested = SR_collection.count_documents(filter=filter_requested)

sum_case_age_days = 0
for SR in SR_collection.find(filter=filter_requested):
    if 'case_age_days' in SR:
        sum_case_age_days += int(SR["case_age_days"])

total_closed = SR_collection.count_documents(filter=filter_closed)



In [ ]:
print(dt_start_utc, dt_end_utc)
print(total_requested, total_closed)
print(sum_case_age_days)

In [ ]:
            update_doc = summary_collection.find_one_and_update(
                {'year' : year},
                {'$set':
                    {
                        "total_requested":int(total_requested),
                        "total_closed":int(total_closed),
                        "percent_closed": float(total_closed/total_requested),
                        "average_case_age_days": float(sum_case_age_days/total_requested)
                        
                    }
                    
                },upsert=True
            )

In [ ]:
def updateDistricts(district):
    filter_requested ={'council_district': district}
    filter_closed ={'council_district': district, 'status':"Closed"}
    
    total_requested = SR_collection.count_documents(filter=filter_requested)

    sum_case_age_days = 0
    for SR in SR_collection.find(filter=filter_requested):
        if 'case_age_days' in SR:
            sum_case_age_days += int(SR["case_age_days"])

    total_closed = SR_collection.count_documents(filter=filter_closed)
    
    update_doc = summary_collection.find_one_and_update(
                {'district' : district},
                {'$set':
                    {
                        "total_requested":int(total_requested),
                        "total_closed":int(total_closed),
                        "percent_closed": float(total_closed/total_requested),
                        "average_case_age_days": float(sum_case_age_days/total_requested)
                        
                    }
                    
                },upsert=True
            )
    
districts = [1,2,3,4,5,6,7,8,9]
for district in districts:
    updateDistricts(district)

In [ ]:
filter={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}

result = client['getitdone_db']['sandiego'].find(filter=filter)

In [ ]:
i = 0
date_requested_count = {}  
date_closed_count = {}

for SR in result:
    
    if 'date_requested' in SR:            
        date_requested_month = SR['date_requested'].month     
        if (date_requested_month in date_requested_count):
            date_requested_count[date_requested_month] += 1        
        else:
            date_requested_count[date_requested_month] = 1            
    if 'date_closed' in SR:
        date_closed_month = SR['date_closed'].month     
        if (date_closed_month in date_closed_count):
            date_closed_count[date_closed_month] += 1        
        else:
            date_closed_count[date_closed_month] = 1 
    
    # SR['case_age_days']
    # SR['service_name']
    # SR['case_record_type']    
    # SR['status']
    # SR['lat']
    # SR['lng']
    # SR['street_address']
    # SR['council_district']
    # SR['comm_plan_code']
    # SR['comm_plan_name']
    # SR['case_origin']
    # SR['public_description']   

    # i += 1
    # if i > 5:
    #     break

In [ ]:
date_requested_count


In [ ]:
date_closed_count

In [8]:
def get_sd_api_data(sd_api_url):
    print("import SR from get it done api")        
    response = requests.get(sd_api_url)
    response_json = response.json()  

    count_insert_SR = 0
    count_update_SR = 0

    for SR in response_json:
        public_description = ""
        media_url = ""

        # Get the data from the results
        service_request_id = int(SR["service_request_id"])
        date_requested_string = SR["requested_datetime"] # or any date sting of differing formats.
        date_requested = parser.parse(date_requested_string)
        updated_datetime_string = SR["updated_datetime"] # or any date sting of differing formats.
        updated_datetime = parser.parse(updated_datetime_string)        
        
        if "description" in SR:
            public_description = SR["description"]
        if 'media_url' in SR:
            media_url = SR["media_url"]

        if SR_collection.count_documents({'service_request_id': service_request_id}) == 0:
            print("Insert new service request! ")            
            insert_doc = { 
                "service_request_id": service_request_id,
                "date_requested": date_requested,
                "date_updated": updated_datetime,
                "status": SR["status"],
                "service_code": SR["service_code"],
                "service_name": SR["service_name"],
                "public_description": public_description,
                "street_address": SR["address"],
                "lat": float(SR["lat"]),
                "lng": float(SR["long"]),
                "media_url": media_url
                }
            doc = SR_collection.insert_one(insert_doc)
            count_insert_SR += 1
            # if config.debug:
            #     print(doc)

        else: 
            print(f"SR {service_request_id} already exists, update existing service request!")
            update_doc = SR_collection.find_one_and_update(
                {'service_request_id' : service_request_id},
                {'$set':
                    {
                        "date_requested": date_requested,
                        "date_updated": updated_datetime,
                        "status": SR["status"],
                        "service_code": SR["service_code"],
                        "service_name": SR["service_name"],
                        "public_description": public_description,
                        "street_address": SR["address"],
                        "lat": float(SR["lat"]),
                        "lng": float(SR["long"]),
                        "media_url": media_url
                    }
                    
                },upsert=True
            )
            count_update_SR += 1
            # if config.debug:
            #     print(update_doc)

    return f"{str(count_insert_SR)} SR created. {str(count_update_SR)} SR updated."

            

In [29]:
def get_sd_csv_data(csv_file):
    print("import SR from get it done historical")        
    i = 0
    columns = []

    with open(csv_file,encoding ="utf8") as csvfile:
        # CSV reader specifies delimiter and variable that holds contents
        reader = csv.reader(csvfile, delimiter=',')

        # print(reader)

        # Read the header row first (skip this step if there is now header)
        csv_header = next(reader)
        # # print(f"CSV Header: {csv_header}")

        count_insert_SR = 0
        count_update_SR = 0

        # Read each row of data after the header
        for SR in reader:
            # build SR dict from csv row
            if i != 0:                 
                dict = {}
                j = 0
                # loop through all headers and create a key value pair and add to dict
                for k in csv_header:
                    dict[k] = SR[j]
                    j += 1
                
                # print(dict)
                
                service_request_id = int(dict["service_request_id"])
                date_requested_string = dict["date_requested"] # or any date sting of differing formats.
                date_requested = parser.parse(date_requested_string)
                
    
                # print(service_request_id)
                                      
                if SR_collection.count_documents({'service_request_id': service_request_id}) == 0:
                    # print("Insert new service request! ")            
                    insert_doc = { 
                        "service_request_id": service_request_id                        
                        }
                    doc = SR_collection.insert_one(insert_doc)                

                    count_insert_SR += 1



                    # if config.debug:
                    #     print(doc)

            
                # print(f"SR {service_request_id} already exists, update existing service request!")
                update_doc = SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "date_requested": date_requested,
                            "case_age_days": int(dict["case_age_days"]),
                            "service_name":	dict["service_name"],                        
                            "status": dict["status"],
                            "street_address": dict["street_address"],
                            "zipcode":	dict["zipcode"],
                            "comm_plan_name": dict["comm_plan_name"],
                            "park_name":dict["park_name"],
                            "case_origin":dict["case_origin"],
                            "referred":	dict["referred"],
                            "public_description": dict["public_description"],
                            "iamfloc":dict["iamfloc"],
                            "floc":dict["floc"]
                        }
                        
                    },upsert=True
                )
                try:
                    date_closed_string = dict["date_closed"] # or any date sting of differing formats.
                    date_closed = parser.parse(date_closed_string)
                    SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "date_closed": date_closed
                        }                            
                    },upsert=True)
                except:
                    pass

                try:
                    case_age_days = int(dict["case_age_days"])   
                    SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "case_age_days": case_age_days
                        }                            
                    },upsert=True)
                except:
                    pass 

                try:
                    lat = float(dict["lat"])   
                    SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "lat": lat
                        }                            
                    },upsert=True)
                except:
                    pass                     
                try:
                    lng = float(dict["lng"])   
                    update_closed_doc = SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "lng":lng
                        }                            
                    },upsert=True)
                except:
                    pass  
                try:
                    council_district = int(dict["council_district"])   
                    update_closed_doc = SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "council_district":council_district
                        }                            
                    },upsert=True)
                except:
                    pass  
                try:
                    comm_plan_code = int(dict["comm_plan_code"])   
                    update_closed_doc = SR_collection.find_one_and_update(
                    {'service_request_id' : service_request_id},
                    {'$set':
                        {
                            "comm_plan_code":comm_plan_code
                        }                            
                    },upsert=True)
                except:
                    pass  
                                                            
                count_update_SR += 1
                # if config.debug:
                #     print(update_doc)

                print(f"SR# {service_request_id} | {str(count_insert_SR)} SR created. {str(count_update_SR)} SR updated.")                   
                count_update_SR += 1
                # if config.debug:
                #     print(update_doc)
            i += 1
            # if i > 10:
            #     break

    return f"csv data imported. {str(count_insert_SR)} SR created. {str(count_update_SR)} SR updated."

            

In [30]:
summary_csv = get_sd_csv_data("C:\Git\git-it-on\static\data\get_it_done_2021_requests_datasd_v1fromMarch.csv")
print(summary_csv)
    

import SR from get it done historical
SR# 3295472 | 0 SR created. 1 SR updated.
SR# 3295473 | 1 SR created. 3 SR updated.
SR# 3295474 | 2 SR created. 5 SR updated.
SR# 3295475 | 3 SR created. 7 SR updated.
SR# 3295476 | 4 SR created. 9 SR updated.
SR# 3295477 | 5 SR created. 11 SR updated.
SR# 3295478 | 6 SR created. 13 SR updated.
SR# 3295479 | 7 SR created. 15 SR updated.
SR# 3295480 | 8 SR created. 17 SR updated.
SR# 3295481 | 9 SR created. 19 SR updated.
SR# 3295482 | 10 SR created. 21 SR updated.
SR# 3295483 | 11 SR created. 23 SR updated.
SR# 3295484 | 12 SR created. 25 SR updated.
SR# 3295485 | 13 SR created. 27 SR updated.
SR# 3295486 | 14 SR created. 29 SR updated.
SR# 3295487 | 15 SR created. 31 SR updated.
SR# 3295488 | 16 SR created. 33 SR updated.
SR# 3295489 | 17 SR created. 35 SR updated.
SR# 3295490 | 18 SR created. 37 SR updated.
SR# 3295491 | 19 SR created. 39 SR updated.
SR# 3295492 | 20 SR created. 41 SR updated.
SR# 3295493 | 21 SR created. 43 SR updated.
SR# 32954

In [ ]:
summary = get_sd_api_data("http://san-diego.spotreporters.com/open311/v2/requests.json")

In [ ]:
print(summary)